Setup code

In [1]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY in .env file
# UPSTAGE_API_KEY=your_api_key

import warnings
warnings.filterwarnings("ignore")

In [2]:
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

query = f"""
Make 3 questions using this text.

{text}
"""

query2 = """
Linear transformation is a function that satisfies two features.
One is homogenity, and other is additivity.

Linear transformation of the subspace is also subspace.
Prove or disprove this statement based on given contents.
"""

query3 = """
The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

In [4]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
In this query, separate the instruction and context. Do not answer to the query, just separate it.
Context is can be a base information or some text to convert or manipulate.
Context can be not provided(N/A for this case).
Instruction and context must not have duplicate contents.
Print instruction as first paragraph and context as second paragraph.
---
Query : {query}
"""
)
chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"query": query3})
print(answer)

Instruction:
The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

Context:
N/A


In [22]:
# answer를 regex로 분리
# instruction을 단계로 분리 (LLM 도움)
# 각 chain을 실행하고 결과 누적으로 얻음
# (전체 히스토리 누적 후) 최종 결과 질문
instr = "Calculate the total number of apples the cafeteria has after using some and buying more."
context = "The cafeteria had 23 apples, used 20 to make lunch, and bought 6 more."

prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", give me what to do to achieve this instruction.
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context})
print(answer)

To calculate the total number of apples the cafeteria has after using some and buying more, follow these steps:

1. Determine the initial number of apples the cafeteria had.
2. Subtract the number of apples used from the initial number to find out how many are left.
3. Add the number of apples bought to the remaining number to find the total number of apples after the purchase.

Here's a simple formula to help you calculate this:

Total apples = Initial apples - Used apples + Bought apples

By following these steps and using the formula, you will be able to calculate the total number of apples the cafeteria has after using some and buying more.
